<a href="https://colab.research.google.com/github/zelal-Eizaldeen/DLH_Project_HurtfulWords_Paper/blob/main/2_Cleaning_data_z.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Import modules
import sys
sys.path.append('/content/drive/MyDrive/MastersDegree/DLH/Project/HurtfulWords/Payel-DLH-related/Colab Notebooks')

In [3]:
import Constants # This module from Hurtful github base code

In [6]:
import pandas as pd
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch.nn.functional as F
from sklearn.model_selection import KFold


In [5]:
PATH_TO_SAMLED_CSVs='/content/drive/MyDrive/MastersDegree/DLH/Project/HurtfulWords/Payel-DLH-related/DataFiles/sampled_files'

In [8]:
# Load all necessary CSV files
patients_df = pd.read_csv(f"{PATH_TO_SAMLED_CSVs}/sampled_PATIENTS.csv")  # Patient info
admissions_df = pd.read_csv(f"{PATH_TO_SAMLED_CSVs}/sampled_ADMISSIONS.csv")  # Admissions info
icds = pd.read_csv(f"{PATH_TO_SAMLED_CSVs}/sampled_DIAGNOSES_ICD.csv")  # Diagnoses information
noteevents_df = pd.read_csv(f"{PATH_TO_SAMLED_CSVs}/sampled_NOTEEVENTS.csv")  # Notes text
icustays = pd.read_csv(f"{PATH_TO_SAMLED_CSVs}/sampled_ICUSTAYS.csv")  # ICU stays info/


In [9]:
icustays.columns=icustays.columns.str.lower()

In [10]:
icustays = icustays.set_index(['subject_id', 'hadm_id'])

In [11]:
patients_df

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,241,257,F,2031-04-03 00:00:00,2121-07-08 00:00:00,2121-07-08 00:00:00,2121-07-08 00:00:00,1
1,249,266,F,2090-12-17 00:00:00,NaN,NaN,NaN,0
2,642,679,F,2059-11-04 00:00:00,2145-03-19 00:00:00,2145-03-19 00:00:00,2145-03-19 00:00:00,1
3,655,695,F,2093-05-14 00:00:00,2178-09-16 00:00:00,NaN,2178-09-16 00:00:00,1
4,674,717,F,2038-05-22 00:00:00,2119-03-22 00:00:00,NaN,2119-03-22 00:00:00,1
...,...,...,...,...,...,...,...,...
2321,31746,43715,F,2160-02-07 00:00:00,NaN,NaN,NaN,0
2322,31751,43732,M,2135-09-29 00:00:00,NaN,NaN,NaN,0
2323,31760,43749,F,1811-11-23 00:00:00,NaN,NaN,NaN,0
2324,31794,43909,F,2073-11-22 00:00:00,2152-10-09 00:00:00,2152-10-09 00:00:00,2152-10-09 00:00:00,1


In [12]:
# Split patients into K folds
n_splits = 12
patients_df = patients_df.sample(frac=1, random_state=42).reset_index(drop=True)
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
for c, i in enumerate(kf.split(patients_df, groups=patients_df.GENDER)):
    patients_df.loc[i[1], 'fold'] = str(c)

patients_df.head()  # Show first 5 rows

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:86: UserWarning: The groups parameter is ignored by KFold
  warnings.warn(


,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,fold
0,36218,60408,M,2106-09-15 00:00:00,NaN,NaN,NaN,0,8
1,15194,16071,F,2088-09-21 00:00:00,2156-12-16 00:00:00,NaN,2156-12-16 00:00:00,1,11
2,32617,46947,M,2049-05-28 00:00:00,NaN,NaN,NaN,0,4
3,37205,64188,M,2140-09-16 00:00:00,NaN,NaN,NaN,0,6
4,33710,51108,M,2123-05-28 00:00:00,NaN,NaN,NaN,0,10


In [13]:
# Merge patients and admissions data
df = pd.merge(patients_df, admissions_df, on='SUBJECT_ID', how='inner')


In [14]:
# Merge death information into a single column
def merge_death(row):
    if not pd.isnull(row.DEATHTIME):
        return row.DEATHTIME
    else:
        return row.DOD
df['DOD_MERGED'] = df.apply(merge_death, axis=1)


In [15]:
df.head()  # Show first 5 rows

,ROW_ID_x,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,fold,ROW_ID_y,...,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED
0,36218,60408,M,2106-09-15 00:00:00,NaN,NaN,NaN,0,8,46403,...,ENGL,NOT SPECIFIED,MARRIED,WHITE,2184-04-13 13:15:00,2184-04-13 18:28:00,ALTERED MENTAL STATUS,0,1,NaN
1,15194,16071,F,2088-09-21 00:00:00,2156-12-16 00:00:00,NaN,2156-12-16 00:00:00,1,11,19618,...,NaN,PROTESTANT QUAKER,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,2156-12-16 00:00:00
2,32617,46947,M,2049-05-28 00:00:00,NaN,NaN,NaN,0,4,41985,...,ENGL,NOT SPECIFIED,DIVORCED,PORTUGUESE,NaN,NaN,CEREBRAL ANIOGRAM,0,1,NaN
3,37205,64188,M,2140-09-16 00:00:00,NaN,NaN,NaN,0,6,47588,...,ENGL,NOT SPECIFIED,NaN,UNKNOWN/NOT SPECIFIED,2176-10-14 11:24:00,2176-10-14 15:05:00,"OVERDOSE, SEPSIS",0,1,NaN
4,33710,51108,M,2123-05-28 00:00:00,NaN,NaN,NaN,0,10,43313,...,PTUN,NOT SPECIFIED,NaN,UNABLE TO OBTAIN,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1,NaN


In [16]:
# Process notes data
noteevents_df = noteevents_df[~(pd.isnull(noteevents_df['HADM_ID']))]  # Drop outpatients (no admission data)
df = pd.merge(left=noteevents_df, right=df, on='HADM_ID', how='left')
df

,ROW_ID,SUBJECT_ID_x,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,UNOBTAINABLE,UNKNOWN (DEFAULT),UNKNOWN/NOT SPECIFIED,2151-07-16 04:49:00,2151-07-16 14:20:00,RT LOWER LOBE PNEUMONIA,0,1,NaN
1,236,26601,116471.0,2127-07-30,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,CATHOLIC,WIDOWED,WHITE,2127-07-21 12:59:00,2127-07-21 20:57:00,CHOLECITHIASIS,0,1,2132-04-27 00:00:00
2,237,26601,155131.0,2131-12-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,ENGL,CATHOLIC,WIDOWED,WHITE,2131-12-23 16:14:00,2131-12-23 19:45:00,PNEUMONIA,0,1,2132-04-27 00:00:00
3,210,21449,139542.0,2146-07-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,PROTESTANT QUAKER,MARRIED,WHITE,NaN,NaN,CHEST PAIN\CATH,0,1,NaN
4,8,31502,125483.0,2174-06-09,NaN,NaN,Discharge summary,Report,NaN,NaN,...,ENGL,JEWISH,MARRIED,WHITE,2174-05-29 03:04:00,2174-05-29 05:52:00,RESPIRATORY FAILURE;CONGESTIVE HEART FAILURE,0,1,2174-08-31 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7886,213134,4225,110284.0,2136-06-03,NaN,NaN,ECG,Report,NaN,NaN,...,NaN,NOT SPECIFIED,SINGLE,WHITE,2136-06-03 11:32:00,2136-06-03 15:05:00,LITHIUM/TCA OD,0,1,NaN
7887,212638,25560,156143.0,2154-12-04,NaN,NaN,ECG,Report,NaN,NaN,...,NaN,PROTESTANT QUAKER,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,NEWBORN,0,1,NaN
7888,213331,55700,150590.0,2163-02-19,NaN,NaN,ECG,Report,NaN,NaN,...,ENGL,OTHER,WIDOWED,WHITE,2163-02-18 16:40:00,2163-02-18 21:19:00,PNEUMONIA,0,1,NaN
7889,213132,4225,110284.0,2136-06-04,NaN,NaN,ECG,Report,NaN,NaN,...,NaN,NOT SPECIFIED,SINGLE,WHITE,2136-06-03 11:32:00,2136-06-03 15:05:00,LITHIUM/TCA OD,0,1,NaN


In [17]:
# Fill missing ethnicity data and clean ethnicity values
df['ETHNICITY'].fillna(value='UNKNOWN/NOT SPECIFIED', inplace=True)

<ipython-input-17-732f9f229723>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ETHNICITY'].fillna(value='UNKNOWN/NOT SPECIFIED', inplace=True)


In [18]:
def cleanField(string):
    mappings = {'HISPANIC OR LATINO': 'HISPANIC/LATINO',
                'BLACK/AFRICAN AMERICAN': 'BLACK',
                'UNABLE TO OBTAIN': 'UNKNOWN/NOT SPECIFIED',
                'PATIENT DECLINED TO ANSWER': 'UNKNOWN/NOT SPECIFIED'}
    bases = ['WHITE', 'UNKNOWN/NOT SPECIFIED', 'BLACK', 'HISPANIC/LATINO', 'OTHER', 'ASIAN']

    if string in bases:
        return string
    elif string in mappings:
        return mappings[string]
    else:
        for i in bases:
            if i in string:
                return i
        return 'OTHER'

df['ETHNICITY_TO_USE'] = df['ETHNICITY'].apply(cleanField)


In [19]:
#Let's see the type of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7891 entries, 0 to 7890
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ROW_ID                7891 non-null   int64  
 1   SUBJECT_ID_x          7891 non-null   int64  
 2   HADM_ID               7891 non-null   float64
 3   CHARTDATE             7891 non-null   object 
 4   CHARTTIME             0 non-null      float64
 5   STORETIME             0 non-null      float64
 6   CATEGORY              7891 non-null   object 
 7   DESCRIPTION           7891 non-null   object 
 8   CGID                  0 non-null      float64
 9   ISERROR               0 non-null      float64
 10  TEXT                  7891 non-null   object 
 11  ROW_ID_x              7891 non-null   int64  
 12  SUBJECT_ID_y          7891 non-null   int64  
 13  GENDER                7891 non-null   object 
 14  DOB                   7891 non-null   object 
 15  DOD                  

In [20]:
# Convert 'chartdate' and 'dob' to datetime type
df['CHARTDATE'] = pd.to_datetime(df['CHARTDATE'])
df['DOB'] = pd.to_datetime(df['DOB'])
df['DOD'] = pd.to_datetime(df['DOD'])



In [21]:
# Ensure that chartdate is later than dob (birthdate)
df = df[df.CHARTDATE >= df.DOD]

In [22]:
# Calculate age based on dob and chartdate

ages = [(df.CHARTDATE.iloc[i] - df.DOD.iloc[i]).days / 365.24 for i in range(df.shape[0])]
df['AGE'] = ages
df.DOD.iloc[300]

<ipython-input-22-eec76ec1b999>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AGE'] = ages


Timestamp('2128-11-11 00:00:00')

Explanation <br>

'fold' = 'NA':
For the rows where the condition df.CATEGORY.isin([...]) is True, the code assigns the value 'NA' to the fold column.
This means that the fold column will be set to 'NA' for rows where the CATEGORY is one of the specified categorie

In [23]:
# Exclude certain categories from fold assignment
df.loc[df.CATEGORY.isin(['Discharge summary', 'Echo', 'ECG']), 'fold'] = 'NA'

In [24]:
# Merge diagnoses data into the main dataframe
icds = icds.groupby('HADM_ID').agg({'ICD9_CODE': lambda x: list(x.values)}).reset_index()
df = pd.merge(left=df, right=icds, on='HADM_ID')

df

,ROW_ID,SUBJECT_ID_x,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA,DOD_MERGED,ETHNICITY_TO_USE,AGE,ICD9_CODE
0,37,32779,133757.0,2167-10-27,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,NaN,NaN,ACUTE SHORTNESS OF BREATH;CONGESTIVE HEART FAI...,1,1,2167-10-27 12:00:00,WHITE,0.0,"[51881, 41071, 5849, 2762, 4280, 4275, 486, 49..."
1,444,15159,187095.0,2103-08-19,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,2103-08-16 21:37:00,2103-08-17 00:04:00,PNEUMONIA,1,1,2103-08-19 17:43:00,WHITE,0.0,"[486, 2765, 29900, 7580]"
2,753,82122,100504.0,2175-02-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,2175-01-29 13:31:00,2175-01-29 15:48:00,CONGESTIVE HEART FAILURE,1,1,2175-02-04 17:50:00,WHITE,0.0,"[41091, 42823, 5849, 4271, 2761, 2762, V667, 4..."
3,494,16447,163590.0,2191-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,...,UNKNOWN/NOT SPECIFIED,NaN,NaN,S/P CARDIAC ARREST\CATH,1,1,2191-09-03 18:45:00,UNKNOWN/NOT SPECIFIED,0.0,"[4275, 4822, 3481, 2967, V182]"
4,908,31778,186882.0,2101-07-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,WHITE,2101-07-08 13:06:00,2101-07-08 17:53:00,ABDOMINAL PAIN,1,1,2101-07-29 18:20:00,WHITE,0.0,"[0383, 51881, 42831, 00845, 78552, 5119, 5849,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,194097,5249,124041.0,2176-12-22,NaN,NaN,ECG,Report,NaN,NaN,...,WHITE,2176-12-22 14:01:00,2176-12-22 16:05:00,MULTIPLE TRAUMA,1,1,2176-12-22 18:44:00,WHITE,0.0,"[85205, 80708, 80502, 8082, 8080, 9587, 9010, ..."
299,171316,19669,168900.0,2128-11-11,NaN,NaN,ECG,Report,NaN,NaN,...,UNKNOWN/NOT SPECIFIED,NaN,NaN,MYOCARDIAL INFARCTION',1,1,2128-11-11 22:47:00,UNKNOWN/NOT SPECIFIED,0.0,"[41051, 4275, 4240, 5789, 42731, 3481, 2762, 5..."
300,171317,19669,168900.0,2128-11-11,NaN,NaN,ECG,Report,NaN,NaN,...,UNKNOWN/NOT SPECIFIED,NaN,NaN,MYOCARDIAL INFARCTION',1,1,2128-11-11 22:47:00,UNKNOWN/NOT SPECIFIED,0.0,"[41051, 4275, 4240, 5789, 42731, 3481, 2762, 5..."
301,171318,19669,168900.0,2128-11-11,NaN,NaN,ECG,Report,NaN,NaN,...,UNKNOWN/NOT SPECIFIED,NaN,NaN,MYOCARDIAL INFARCTION',1,1,2128-11-11 22:47:00,UNKNOWN/NOT SPECIFIED,0.0,"[41051, 4275, 4240, 5789, 42731, 3481, 2762, 5..."


In [26]:
# Map language codes to human-readable categories
def map_lang(x):
    if x == 'ENGL':
        return 'English'
    if pd.isnull(x):
        return 'Missing'
    return 'Other'
df['LANGUAGE_TO_USE'] = df['LANGUAGE'].apply(map_lang)


In [27]:
#Convert Columns into lower letters
df.columns = df.columns.str.lower()

In [28]:

# Check that all required columns from Constants are present in the dataframe
for i in Constants.groups:
    assert(i['name'] in df.columns), i['name']

In [ ]:
# This is not applied as we don't have this table
# Load ICU stays data
# acuities = pd.read_csv('ACUITY.csv')

# # Merge acuity data into the main dataframe
# df = pd.merge(df, acuities.drop(columns=['subject_id', 'hadm_id']), on='icustay_id', how='left')

In [29]:
def fill_icustay(row):
    try:
        opts = icustays.xs((row['subject_id'], row['hadm_id']), drop_level=False)
    except KeyError:  # Handle cases where the index is not found
        return None

    # Determine the charttime
    if pd.isnull(row['charttime']):
        charttime = row['chartdate'] + pd.Timedelta(days=2)
    else:
        charttime = row['charttime']

    # Filter ICU stays based on intime
    stay = opts[opts['intime'] <= charttime].sort_values(by='intime', ascending=True)

    if len(stay) == 0:
        return None

    return stay.iloc[-1]['icustay_id']

# Apply the function to filter specific categories
df['icustay_id'] = df[df.category.isin(['Discharge summary', 'Physician', 'Nursing', 'Nursing/other'])].apply(fill_icustay, axis=1)


In [30]:
df.head()

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,dod_merged,ethnicity_to_use,age,icd9_code,language_to_use,icustay_id
0,37,32779,133757.0,2167-10-27,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,ACUTE SHORTNESS OF BREATH;CONGESTIVE HEART FAI...,1,1,2167-10-27 12:00:00,WHITE,0.0,"[51881, 41071, 5849, 2762, 4280, 4275, 486, 49...",Other,None
1,444,15159,187095.0,2103-08-19,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2103-08-17 00:04:00,PNEUMONIA,1,1,2103-08-19 17:43:00,WHITE,0.0,"[486, 2765, 29900, 7580]",Missing,None
2,753,82122,100504.0,2175-02-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2175-01-29 15:48:00,CONGESTIVE HEART FAILURE,1,1,2175-02-04 17:50:00,WHITE,0.0,"[41091, 42823, 5849, 4271, 2761, 2762, V667, 4...",Other,None
3,494,16447,163590.0,2191-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,S/P CARDIAC ARREST\CATH,1,1,2191-09-03 18:45:00,UNKNOWN/NOT SPECIFIED,0.0,"[4275, 4822, 3481, 2967, V182]",Missing,None
4,908,31778,186882.0,2101-07-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2101-07-08 17:53:00,ABDOMINAL PAIN,1,1,2101-07-29 18:20:00,WHITE,0.0,"[0383, 51881, 42831, 00845, 78552, 5119, 5849,...",English,None


In [ ]:
#This didn't work and we used ChatGPT to know the code that may work, which is in the cell above
# def fill_icustay(row):
#     opts = icustays.loc[[row['subject_id'],row['hadm_id']]]
#     if pd.isnull(row['charttime']):
#         charttime = row['chartdate'] + pd.Timedelta(days = 2)
#     else:
#         charttime = row['charttime']
#     stay = opts[(opts['intime'] <= charttime)].sort_values(by = 'intime', ascending = True)

#     if len(stay) == 0:
#         return None
#         #print(row['subject_id'], row['hadm_id'], row['category'])
#     return stay.iloc[-1]['icustay_id']

# df['icustay_id'] = df[df.category.isin(['Discharge summary','Physician ','Nursing','Nursing/other'])].apply(fill_icustay, axis = 1)

KeyError: 'subject_id'

In [31]:
# Fill ages over 90 with 91.4 (assuming this is to standardize)  #ned to check with Zilal
df.loc[df.age >= 90, 'age'] = 91.4
df

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,dod_merged,ethnicity_to_use,age,icd9_code,language_to_use,icustay_id
0,37,32779,133757.0,2167-10-27,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,ACUTE SHORTNESS OF BREATH;CONGESTIVE HEART FAI...,1,1,2167-10-27 12:00:00,WHITE,0.0,"[51881, 41071, 5849, 2762, 4280, 4275, 486, 49...",Other,None
1,444,15159,187095.0,2103-08-19,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2103-08-17 00:04:00,PNEUMONIA,1,1,2103-08-19 17:43:00,WHITE,0.0,"[486, 2765, 29900, 7580]",Missing,None
2,753,82122,100504.0,2175-02-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2175-01-29 15:48:00,CONGESTIVE HEART FAILURE,1,1,2175-02-04 17:50:00,WHITE,0.0,"[41091, 42823, 5849, 4271, 2761, 2762, V667, 4...",Other,None
3,494,16447,163590.0,2191-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,S/P CARDIAC ARREST\CATH,1,1,2191-09-03 18:45:00,UNKNOWN/NOT SPECIFIED,0.0,"[4275, 4822, 3481, 2967, V182]",Missing,None
4,908,31778,186882.0,2101-07-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2101-07-08 17:53:00,ABDOMINAL PAIN,1,1,2101-07-29 18:20:00,WHITE,0.0,"[0383, 51881, 42831, 00845, 78552, 5119, 5849,...",English,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,194097,5249,124041.0,2176-12-22,NaN,NaN,ECG,Report,NaN,NaN,...,2176-12-22 16:05:00,MULTIPLE TRAUMA,1,1,2176-12-22 18:44:00,WHITE,0.0,"[85205, 80708, 80502, 8082, 8080, 9587, 9010, ...",Missing,NaN
299,171316,19669,168900.0,2128-11-11,NaN,NaN,ECG,Report,NaN,NaN,...,NaN,MYOCARDIAL INFARCTION',1,1,2128-11-11 22:47:00,UNKNOWN/NOT SPECIFIED,0.0,"[41051, 4275, 4240, 5789, 42731, 3481, 2762, 5...",Missing,NaN
300,171317,19669,168900.0,2128-11-11,NaN,NaN,ECG,Report,NaN,NaN,...,NaN,MYOCARDIAL INFARCTION',1,1,2128-11-11 22:47:00,UNKNOWN/NOT SPECIFIED,0.0,"[41051, 4275, 4240, 5789, 42731, 3481, 2762, 5...",Missing,NaN
301,171318,19669,168900.0,2128-11-11,NaN,NaN,ECG,Report,NaN,NaN,...,NaN,MYOCARDIAL INFARCTION',1,1,2128-11-11 22:47:00,UNKNOWN/NOT SPECIFIED,0.0,"[41051, 4275, 4240, 5789, 42731, 3481, 2762, 5...",Missing,NaN


In [32]:
# Saving the final dataframe as a pickle for future use
df.to_pickle(f"{PATH_TO_SAMLED_CSVs}/df_raw.pkl")

In [33]:

# Display the first few rows of the dataframe
display(df.head())

,row_id,subject_id_x,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,...,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data,dod_merged,ethnicity_to_use,age,icd9_code,language_to_use,icustay_id
0,37,32779,133757.0,2167-10-27,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,ACUTE SHORTNESS OF BREATH;CONGESTIVE HEART FAI...,1,1,2167-10-27 12:00:00,WHITE,0.0,"[51881, 41071, 5849, 2762, 4280, 4275, 486, 49...",Other,None
1,444,15159,187095.0,2103-08-19,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2103-08-17 00:04:00,PNEUMONIA,1,1,2103-08-19 17:43:00,WHITE,0.0,"[486, 2765, 29900, 7580]",Missing,None
2,753,82122,100504.0,2175-02-04,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2175-01-29 15:48:00,CONGESTIVE HEART FAILURE,1,1,2175-02-04 17:50:00,WHITE,0.0,"[41091, 42823, 5849, 4271, 2761, 2762, V667, 4...",Other,None
3,494,16447,163590.0,2191-09-03,NaN,NaN,Discharge summary,Report,NaN,NaN,...,NaN,S/P CARDIAC ARREST\CATH,1,1,2191-09-03 18:45:00,UNKNOWN/NOT SPECIFIED,0.0,"[4275, 4822, 3481, 2967, V182]",Missing,None
4,908,31778,186882.0,2101-07-29,NaN,NaN,Discharge summary,Report,NaN,NaN,...,2101-07-08 17:53:00,ABDOMINAL PAIN,1,1,2101-07-29 18:20:00,WHITE,0.0,"[0383, 51881, 42831, 00845, 78552, 5119, 5849,...",English,None
